In [1]:
import boto3
import pandas as pd
import json
import sklearn.metrics as metrics

In [2]:
# nome do endpoint - verificar o resultado de predictor.endpoint_name
endpoint_name = 'datascience-credito-xgboost'

In [3]:
runtime = boto3.client('runtime.sagemaker')

In [5]:
dados_teste = pd.read_csv('dados/teste.csv')
alvo = dados_teste['ALVO']

# remover cabecalho e primeira coluna
dados_teste = dados_teste.drop(['ALVO'], axis=1)

dados_teste.to_csv('dados/teste_endpoint.csv', index=False, header=False)
csv_buffer = open('dados/teste_endpoint.csv')
my_payload_as_csv = csv_buffer.read()

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body= my_payload_as_csv,
    ContentType = 'text/csv')

result = response['Body'].read()
result = result.decode("utf-8")
result = result.split(',')
preds = [float((num)) for num in result]
preds = [round(num) for num in preds]

print(metrics.confusion_matrix(alvo, preds))
print(metrics.classification_report(alvo, preds))

[[336 134]
 [108 404]]
              precision    recall  f1-score   support

           0       0.76      0.71      0.74       470
           1       0.75      0.79      0.77       512

    accuracy                           0.75       982
   macro avg       0.75      0.75      0.75       982
weighted avg       0.75      0.75      0.75       982

